In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")

## Query all your positions


In [ ]:
from agent0 import Chain, Hyperdrive
import os

# PUBLIC_ADDRESS = "0xPUBLIC_ADDRESS"
# RPC_URI = "https://sepolia.rpc.url"
PUBLIC_ADDRESS = os.getenv("PUBLIC_ADDRESS")
RPC_URI = os.getenv("RPC_URI")

# Address of Hyperdrive Sepolia registry
REGISTRY_ADDRESS = "0x4ba58147e50e57e71177cfedb1fac0303f216104"

## View open and closed positions in all pools
chain = Chain(RPC_URI)
agent = chain.init_agent(public_address=PUBLIC_ADDRESS)
registered_pools = Hyperdrive.get_hyperdrive_pools_from_registry(
    chain,
    registry_address=REGISTRY_ADDRESS,
)
agent.get_positions(pool_filter=registered_pools, show_closed_positions=True)

## Automate withdrawing funds from matured positions


In [ ]:
PRIVATE_KEY = os.getenv("PRIVATE_KEY")

# Initialize agent with private key for transactions
agent = chain.init_agent(private_key=PRIVATE_KEY)
for pool in registered_pools:
    # Close all mature longs
    for long in agent.get_longs(pool=pool):
        if long.maturity_time > chain.block_time():
            print(f"Close long-{long.maturity_time} on {pool.name}")
            agent.close_long(maturity_time=long.maturity_time, bonds=long.balance, pool=pool)
    # Close all mature shorts
    for short in agent.get_shorts(pool=pool):
        if short.maturity_time > chain.block_time():
            print(f"Close short-{short.maturity_time} on {pool.name}")
            agent.close_short(maturity_time=short.maturity_time, bonds=short.balance, pool=pool)